In [71]:
# Build individual onset file for spm analysis and save it in fmri preprocess folder
# Import
import os
import pandas as pd

# Set path
path_root = "/gs/project/gsf-624-aa/HCP/HCP_raw_data_900R"
path_out = "/gs/project/gsf-624-aa/HCP/fmri_preprocess_all_tasks_niak-fix-scrub_900R"

# Loop over subject and runs and create individual onset
list_subj = os.listdir(path_root)
# loop over runs
for run in 'LR','RL':
    for index, subj in enumerate(list_subj):
        path_eprime = os.path.join(path_root,subj,"unprocessed/3T/tfMRI_LANGUAGE_{}/LINKED_DATA/EPRIME/EVs".format(run))
        if not os.path.exists(path_eprime):
            print('Subject %s has no Eprime folder for LANGUAGE_%s task' % (subj, run))
            continue
        
        # Create subject EVs folder
        eprime_out = os.path.join(path_out,'EVs','HCP{}'.format(subj),'LANGUAGE_{}'.format(run))
        if not os.path.exists(eprime_out):
            os.makedirs(eprime_out)
        
        # Read math eprime files
        eprime_math = pd.read_csv(os.path.join(path_eprime,"math.txt"), sep='\t',header=None)
        
        # Add onset_trial name
        onset_list = ["math".format(row) for ind, row in eprime_math.iterrows()]
        eprime_math.loc[:,'']=onset_list
        
        # re-arrange colomns
        cols = eprime_math.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        eprime_math = eprime_math[cols]
        
        # Read story eprime files
        eprime_story = pd.read_csv(os.path.join(path_eprime,"story.txt"), sep='\t',header=None)
        
        # Add onset_trial name
        onset_list = ["story".format(row) for ind, row in eprime_story.iterrows()]
        eprime_story.loc[:,'']=onset_list
        
        # re-arrange colomns
        cols = eprime_story.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        eprime_story = eprime_story[cols]
        
        # append story to math
        eprime_final = eprime_math.append(eprime_story)
        
        # Put header
        eprime_final.rename(columns={0:'start',1:'duration',2:'repetition'},inplace=True)
        
        # Save file 
        eprime_final.to_csv(os.path.join(eprime_out,"LANGUAGE_{}_spm_onset.csv".format(run)),index=False)
